# Lightning prediction
***
Yang Chen (yang.chen@uci.edu)
> All calculations related to the lightning predictions

***
***


## Preparation

### Import libraries

In [1]:
import numpy as np
import numpy.ma as ma
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap,cm
from matplotlib.pyplot import cm as pltcm
import matplotlib.lines as mlines
import pandas as pd
import xarray as xr
import seaborn as sns
from pandas.plotting import scatter_matrix
from scipy.stats import gaussian_kde
from scipy.optimize import curve_fit,leastsq
import statsmodels.api as sm
import statsmodels.formula.api as smf
from osgeo import gdal
import cmocean
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

### Constants and styles

In [2]:
# constants
latmesh1x1,lonmesh1x1 = np.mgrid[-89.5:89.5:180j,-179.5:179.5:360j]
area = np.tile(np.cos(np.radians(np.linspace(-89.875,89.875,720)))*772.,(1440,1)).T  # km2/0.25deg, [720,1440]
lats = np.linspace(-89.5,89.5,180)
lons = np.linspace(-179.5,179.5,360)

# seaborn styles
sns.set_context('notebook',font_scale=1.5)
sns.set_style('ticks')
yloc=1.05

# CMIP5 models
ModelArray=['CCSM4','CMCC-CM','GFDL-CM3','GFDL-ESM2G','GFDL-ESM2M',
            'GISS-E2-H','GISS-E2-R','HadGEM2-ES','IPSL-CM5A-LR','IPSL-CM5A-MR',
            'MIROC-ESM','MIROC5','MRI-CGCM3','NorESM1-M','bcc-csm1-1']
nmodel = len(ModelArray)

# 1x1 BFS mask 
maskBFS45 = (xr.open_dataarray('/Users/yangchen/Jacaranda/ProjectData/Lightning/maskBFS.nc').values==1)  # North of 45N
maskBFS55 = np.copy(maskBFS45)
maskBFS55[:145,:] = False   # North of 55N

### Read CMIP5 all model mean CAPExP (1996-1999)  from romps data and calculate the summer mean

In [3]:
# subroutine used to read CXP, CAPE, TP during summer of 1996-1999 from Romp's data
def CMIP5read(directory='CAPE/CAPE_historical',variable='CxP',period='historical_1996-1999',dim='m',scl=1):
    # all-model and all-model mean during MJJA
    res='1x1.month.20yrmean'
    vname = variable + 'mon'    
    v, vall= np.zeros((180,360)), np.zeros((180,360,nmodel))
    for imodel in range(nmodel):
        model = ModelArray[imodel]
        ds = xr.open_dataset('/Users/yangchen/Jacaranda/Data/CMIP5/'+directory+'/'+variable+'_'+res+'_'+model+'_'+period+'.nc')
        vall[:,:,imodel] = ds[vname][4:8,:,:].mean(dim=dim).values * scl
        v += vall[:,:,imodel]      
    v /= nmodel    
    return v, vall

In [4]:
# subroutine used to read TAS during summer of 1996-1999 (approximated using pre-calculated 1986-2005 summer means)
def CMIP5readTAS(directory='TAS/tas_Amon_historical',period='historical_1986-2005'):
    vTAS, vTASall= np.zeros((180,360)), np.zeros((180,360,nmodel))   # K
    for imodel in range(nmodel):
        model = ModelArray[imodel]
        dshist = xr.open_dataset('/Users/yangchen/Jacaranda/Data/CMIP5/'+directory+'/tas_0.25.20yrmeanmon_Amon_'+model+'_'+period+'.nc')
        TAShist = dshist.tas[4:8,:,:].mean(dim='mon')
        vTASall[:,:,imodel] = TAShist.values.reshape(180,4,360,4).mean(axis=3).mean(axis=1) # convert to 1x1
        vTAS += vTASall[:,:,imodel]      
    vTAS /= nmodel   
    return vTAS, vTASall

In [5]:
# read CxP, CAPE, and TP
vCxP,vCxPall = CMIP5read()
vCAPE,vCAPEall = CMIP5read(variable='CAPE',period='historical_1996-1999',dim='m')
vTP,vTPall = CMIP5read(variable='TP',period='historical_1996-1999',dim='m',scl=24.*3600) # convert from kg/m2/s to mm/day

# read TAS
vTAS,vTASall = CMIP5readTAS()

### Read observational data that represent present day scenario

In [6]:
imarr = np.array([4,5,6,7,16,17,18,19,28,29,30,31,40,41,42,43])  # month indices for MJJA (1996-1999)
imarrclim = np.array([4,5,6,7]) # month indices for MJJA (climatology)

# T2m from ERA interim
vT2m = ma.average(xr.open_dataarray('/Users/yangchen/Jacaranda/Data/ERAinterim/T2m.mon_1x1_1996-1999.nc')[:,:,imarr],axis=2)
vT2m[:,0]=vT2m[:,1]

# tree cover fraction
vTCF = ( xr.open_dataset('/Users/yangchen/Jacaranda/Data/MCD12C1/MCD12LCTF.IGBP.2012.nc')
         .LCTF[::-1,:,1:6].sum(dim='l').to_masked_array()
         .reshape(180,4,360,4).mean(axis=3).mean(axis=1) )

# elevation
vELEV = ( xr.open_dataset('/Users/yangchen/Jacaranda/Data/GTOPO30/GTOPO30_0p25.nc')
         .Elev.to_masked_array()
         .reshape(180,4,360,4).mean(axis=3).mean(axis=1) )

# GFED Burned area (1996-1999)
vBA = ( (xr.open_dataset('/Users/yangchen/Jacaranda/Data/GFED4s/BurnedArea/BA/nc/BAallveg.nc')
       .BAall[:,:,:].mean(dim='time').to_masked_array()*12/area*100)
       .reshape(180,4,360,4).mean(axis=3).mean(axis=1) )

# LIS/OTD Flash rate (FR)
vFR = ma.average(xr.open_dataarray('/Users/yangchen/Jacaranda/Data/LIS-OTD/HRMC/LISOTD_HRMC_1x1_1996-1999.nc')[:,:,imarrclim],axis=2)

# permafrost soil carbon
PSC = np.zeros((180,360))
PSC[:] = np.nan
PSC[(180-56):,:] = ( xr.open_dataset('/Users/yangchen/Jacaranda/Data/NCSCD/NCSCDv2_Circumpolar_netCDF_1deg/NCSCDv2_Circumpolar_WGS84_SOCC100_1deg.nc')
                    .NCSCDv2[::-1,:]*0.1 )    # unit: convert from hgC/m2 to kg/m2
vPSC = ma.masked_invalid(PSC)                     # mask out invalid values

In [7]:
# now use df_boreal55
maskBFS = maskBFS55.copy()

### Read and record CMIP5 modeled CAPExP for historical (1986-2005) and future (2081-2100) periods

In [8]:
# subroutine used to read PRC during summer of 20 year mean at historical and future periods
def CMIP5readPRC(directory='PRC/prc_Amon_historical',period='historical_1986-2005'):
    vPRC, vPRCall= np.zeros((180,360)), np.zeros((180,360,nmodel))   # K
    for imodel in range(nmodel):
        model = ModelArray[imodel]
        dshist = xr.open_dataset('/Users/yangchen/Jacaranda/Data/CMIP5/'+directory+'/prc_0.25.20yrmeanmon_Amon_'+model+'_'+period+'.nc')
        PRChist = dshist.prc[4:8,:,:].mean(dim='mon')
        vPRCall[:,:,imodel] = PRChist.values.reshape(180,4,360,4).mean(axis=3).mean(axis=1)*(24.*3600) # convert to 1x1, convert from kg/m2/s to mm/day
        vPRC += vPRCall[:,:,imodel]      
    vPRC /= nmodel   
    return vPRC, vPRCall

In [9]:
# read CxP, CAPE, and TP
capehistmean,capehistall = CMIP5read(variable='CAPE',period='historical_1986-2005')    # J/kg
capercpmean,capercpall = CMIP5read(directory='CAPE/CAPE_rcp85',variable='CAPE',period='rcp85_2081-2100')

cxphistmean,cxphistall = CMIP5read(variable='CxP',period='historical_1986-2005')       # W/m2
cxprcpmean,cxprcpall = CMIP5read(directory='CAPE/CAPE_rcp85',variable='CxP',period='rcp85_2081-2100')

tprhistmean,tprhistall = CMIP5read(variable='TP',period='historical_1986-2005',scl=24.*3600)   # kg/m2/s
tprrcpmean,tprrcpall = CMIP5read(directory='CAPE/CAPE_rcp85',variable='TP',period='rcp85_2081-2100',scl=24.*3600)

# read PRC
prchistmean,prchistall = CMIP5readPRC()     # kg/m2/s
prcrcpmean,prcrcpall = CMIP5readPRC(directory='PRC/prc_Amon_rcp85',period='rcp85_2081-2100')

# set tas
tashistmean,tashistall = vTAS, vTASall 
tasrcpmean,tasrcpall = CMIP5readTAS(directory='TAS/tas_Amon_rcp85',period='rcp85_2081-2100')

## Predict future FR

***

### Using the default power law model
using three CAPExP mean values from CMIP5:  
* CxPpd -   summer of 1996-1999 (present day, time period when the regression was derived) 
* CxPhist - summer of 1986-2005 (historical)
* CxPrcp -  summer of 2081-2100 (future)

method:
* Use present day (1996-1999) all model mean CxPpd and observed FR (FRpd) to derive the optimized a,b for FR = a * (CAPExP)^b
* For each CMIP5 model, calculate modeled mean FR for 1986-2005 using the above derived a,b and CxPhist:
    * FRhist_i = a * (CxPhist_i)^b
* Similarly, for each model, calculate modeled mean FR for 2081-2100 using a,b and CxPrcp:
    * FRrcp_i = a * (CxPrcp_i)^b
* To derive the final estimation of FRrcp, we use modeled change (obtained from the all-model mean FRhist and FRrcp) to scale the FR observation in present day:
    * FRrcp_final = FRpd * (FRrcp_mean/FRhist_mean)

output:
* All-model mean predictions: FRfuture_final,FRfuturedC_final,FRfuturedP_final
* FR prediction from each CMIP5 model: FRfuture_final_all
* FR prediction uncertainty range (from different models): vFRscaler_std

In [10]:
# read parameters
vtmp = pd.read_csv('~/GoogleDrive/My/My.Research/UCI/Projects/Lightning/Results/FR.vs.CxP_pl.csv',index_col=0)
a_power,b_power = vtmp['0']['a'],vtmp['0']['b']

In [11]:
# default: powerlaw approach, return more parameters
def Proj_pl(a_power,b_power,cxphistall,cxprcpall,capehistall,capercpall,tprhistall,tprrcpall,vFR):
    # first, apply the powerlaw model to historical and future meterology from each model
    vFRhist_map_all,vFRhist_map_mean = np.zeros((180,360,nmodel)), np.zeros((180,360))
    vFRfuture_map_all,vFRfuture_map_mean = np.zeros((180,360,nmodel)), np.zeros((180,360))
    vFRfuturedC_map_all,vFRfuturedC_map_mean = np.zeros((180,360,nmodel)),np.zeros((180,360))
    vFRfuturedP_map_all,vFRfuturedP_map_mean = np.zeros((180,360,nmodel)),np.zeros((180,360))
    for imodel in range(nmodel):
        model = ModelArray[imodel]        
        # present day
        vFRhist =  a_power*cxphistall[:,:,imodel]**b_power
        vFRhist_map_all[:,:,imodel] = vFRhist[:,:]
        vFRhist_map_mean += vFRhist_map_all[:,:,imodel] 
        # scale approach using CAPExP
        vCxPfuture = cxprcpall[:,:,imodel] 
        vFRfuture =  a_power*vCxPfuture**b_power
        vFRfuture_map_all[:,:,imodel] = vFRfuture[:,:]
        vFRfuture_map_mean += vFRfuture_map_all[:,:,imodel] 
        # scale approach using CAPE
        vCxPfuture = cxphistall[:,:,imodel]*(capercpall[:,:,imodel]/capehistall[:,:,imodel])
        vFRfuture = a_power*vCxPfuture**b_power
        vFRfuturedC_map_all[:,:,imodel] = vFRfuture[:,:]
        vFRfuturedC_map_mean += vFRfuturedC_map_all[:,:,imodel]  
        # scale approach using P
        vCxPfuture = cxphistall[:,:,imodel]*(tprrcpall[:,:,imodel]/tprhistall[:,:,imodel])
        vFRfuture = a_power*vCxPfuture**b_power
        vFRfuturedP_map_all[:,:,imodel] = vFRfuture[:,:]
        vFRfuturedP_map_mean += vFRfuturedP_map_all[:,:,imodel]  
    vFRhist_map_mean /= nmodel
    vFRfuture_map_mean /= nmodel
    vFRfuturedC_map_mean /= nmodel
    vFRfuturedP_map_mean /= nmodel
    
    # normalized uncertainty in scaler FRf/FRh: dscl/mean(scl) = np.sqrt((dFRf/dFRfmean)**2+(dFRh/dFRhmean)**2)
    vFRscaler_map_std = np.zeros((180,360))
    for ix in range(180):
        for iy in range(360):
            if vFRhist_map_mean[ix,iy]>0:
                vFRfuture_std = np.std(vFRfuture_map_all[ix,iy,:])
                vFRhist_std = np.std(vFRhist_map_all[ix,iy,:])
                vFRscaler_map_std[ix,iy] = np.sqrt((vFRhist_std/vFRhist_map_mean[ix,iy])**2+
                                                   (vFRfuture_std/vFRfuture_map_mean[ix,iy])**2)    
    
    # the final projection is using modeled FRfuture/FRhist to scale observed FR
    vFRfuture_final = vFRfuture_map_mean/vFRhist_map_mean*vFR         # use all model mean to do scaling
    vFRfuturedC_final = vFRfuturedC_map_mean/vFRhist_map_mean*vFR
    vFRfuturedP_final = vFRfuturedP_map_mean/vFRhist_map_mean*vFR
    vFRfuture_final_all = np.zeros((180,360,nmodel))
    for imodel in range(nmodel):       # use single model map to do scaling
        model = ModelArray[imodel]
        vFRfuture_final_all[:,:,imodel] = vFRfuture_map_all[:,:,imodel]/vFRhist_map_all[:,:,imodel]*vFR
        
    # return values
    return vFRfuture_final,vFRfuture_final_all,vFRfuturedC_final,vFRfuturedP_final,vFRscaler_map_std

In [12]:
vFRfuture_final,vFRfuture_final_all,vFRfuturedC_final,vFRfuturedP_final,vFRscaler_std = Proj_pl(a_power,b_power,cxphistall,cxprcpall,capehistall,capercpall,tprhistall,tprrcpall,vFR)
vFRfuture_final = ma.masked_array(vFRfuture_final,mask=(maskBFS==False))
vFRscaler_std = ma.masked_array(vFRscaler_std,mask=(maskBFS==False))

### Using the other regression models
only the all-CMIP5-model means are recorded

In [13]:
def Proj_other(mop,vpara,cxphistall,cxprcpall,vFR):
    # first, apply the powerlaw model to historical and future meterology from each model
    vFRhist = np.zeros((180,360))
    vFRfuture = np.zeros((180,360))
    
    for imodel in range(nmodel):
        model = ModelArray[imodel]       
        
        if mop=='pl_op':
            a_powerop,b_powerop = vpara
            vFRhist +=  a_powerop*cxphistall[:,:,imodel]**b_powerop
            vFRfuture +=  a_powerop*cxprcpall[:,:,imodel]**b_powerop
#             print(imodel,vFRfuture.min(),vFRhist.min(),vFRfuture.mean(),vFRhist.mean(),cxphistall[:,:,imodel].mean())
        if mop=='sc':
            a_scale = vpara
            vFRhist +=  cxphistall[:,:,imodel]*a_scale
            vFRfuture +=  cxprcpall[:,:,imodel]*a_scale
        if mop=='li':
            a_linear,b_linear = vpara            
            vFRhist +=  np.clip(a_linear*cxphistall[:,:,imodel]+b_linear,0,None)
            vFRfuture +=  np.clip(a_linear*cxprcpall[:,:,imodel]+b_linear,0,None) 
        if mop=='li2':
            a_linear2,b_linear2 = vpara               
            vFRhist +=  np.clip(a_linear2*cxphistall[:,:,imodel]+b_linear2,0,None)
            vFRfuture +=  np.clip(a_linear2*cxprcpall[:,:,imodel]+b_linear2,0,None)
#             print(imodel,vFRfuture.min(),vFRhist.min(),vFRfuture.mean(),vFRhist.mean())
        if mop=='np':
            binedge,bincenter,Ybinmn,Ybinstd,oobpara = vpara
            xbin,ybin=bincenter[:-1],Ybinmn[:-1]
            
            vFRhist1m = np.zeros((180,360))
            for ix in range(180):
                for iy in range(360):
                    vx = cxphistall[ix,iy,imodel]
                    if vx<xbin[-1]:
                        vFRhist1m[ix,iy] = np.interp(vx,xbin,ybin)
                    else:
                        vFRhist1m[ix,iy] = oobpara[0] + oobpara[1]*vx                    
            vFRhist += vFRhist1m
            
            vFRfuture1m = np.zeros((180,360))
            for ix in range(180):
                for iy in range(360):
                    vx = cxprcpall[ix,iy,imodel]
                    if vx<xbin[-1]:
                        vFRfuture1m[ix,iy] = np.interp(vx,xbin,ybin)
                    else:
                        vFRfuture1m[ix,iy] = oobpara[0] + oobpara[1]*vx                    
            vFRfuture += vFRfuture1m
            
#             binedge,Ybinmn = vpara
#             vFRhist = npara_extrp2d(cxphistall[:,:,imodel],binedge,Ybinmn)
#             vFRfuture = npara_extrp2d(cxprcpall[:,:,imodel],binedge,Ybinmn)        
        
    vFRhist /= nmodel
    vFRfuture /= nmodel   
    
    # the final projection is using modeled FRfuture/FRhist to scale observed FR
    vFRfuture_final = vFRfuture/vFRhist*vFR
    
    if mop=='li' or mop=='li2':
        vFRfuture_final = vFR + (vFRfuture-vFRhist)
        
    # return values
    return vFRfuture_final

apply different models

In [14]:
vtmp = pd.read_csv('~/GoogleDrive/My/My.Research/UCI/Projects/Lightning/Results/FR.vs.CxP_pl_op.csv',index_col=0)
a_power_op,b_power_op = vtmp['0']['a'],vtmp['0']['b']
vFRfuture_pl_op = Proj_other('pl_op',(a_power_op,b_power_op),cxphistall,cxprcpall,vFR)
vFRfuture_pl_op = ma.masked_array(vFRfuture_pl_op,mask=(maskBFS==False))

In [15]:
vtmp = pd.read_csv('~/GoogleDrive/My/My.Research/UCI/Projects/Lightning/Results/FR.vs.CxP_sc.csv',index_col=0)
a_scale = vtmp['0']['a']
vFRfuture_sc = Proj_other('sc',a_scale,cxphistall,cxprcpall,vFR)
vFRfuture_sc = ma.masked_array(vFRfuture_sc,mask=(maskBFS==False))

In [16]:
vtmp = pd.read_csv('~/GoogleDrive/My/My.Research/UCI/Projects/Lightning/Results/FR.vs.CxP_li.csv',index_col=0)
a_linear,b_linear = vtmp['0']['a'],vtmp['0']['b']
vFRfuture_li = Proj_other('li',(a_linear,b_linear),cxphistall,cxprcpall,vFR)
vFRfuture_li = ma.masked_array(vFRfuture_li,mask=(maskBFS==False))

In [17]:
vtmp = pd.read_csv('~/GoogleDrive/My/My.Research/UCI/Projects/Lightning/Results/FR.vs.CxP_li2.csv',index_col=0)
a_linear2,b_linear2 = vtmp['0']['a'],vtmp['0']['b']
vFRfuture_li2 = Proj_other('li2',(a_linear2,b_linear2),cxphistall,cxprcpall,vFR)
vFRfuture_li2 = ma.masked_array(vFRfuture_li2,mask=(maskBFS==False))

In [18]:
vtmp = pd.read_csv('~/GoogleDrive/My/My.Research/UCI/Projects/Lightning/Results/FR.vs.CxP_np.csv',index_col=0)
binedge,bincenter,Ybinmn,Ybinstd,oobpara = vtmp.loc['binedge'].values,vtmp.loc['bincenter'].values,vtmp.loc['Ybinmn'].values,vtmp.loc['Ybinstd'].values,vtmp.loc['oobpara'].values
vFRfuture_np = Proj_other('np',(binedge,bincenter,Ybinmn,Ybinstd,oobpara ),cxphistall,cxprcpall,vFR)
vFRfuture_np = ma.masked_array(vFRfuture_np,mask=(maskBFS==False))

### Save predicted values

In [19]:
ds = xr.Dataset({'vFRfuture_final': (['lat', 'lon'],  vFRfuture_final), 
                 'vFRfuture_final_all': (['lat', 'lon', 'mod'], vFRfuture_final_all),
                 'vFRfuturedC_final': (['lat', 'lon'],  vFRfuturedC_final), 
                 'vFRfuturedP_final': (['lat', 'lon'],  vFRfuturedP_final), 
                 'vFRscaler_std': (['lat', 'lon'],  vFRscaler_std),  
                 'vFRfuture_pl_op': (['lat', 'lon'],  vFRfuture_pl_op), 
                 'vFRfuture_sc': (['lat', 'lon'],  vFRfuture_sc), 
                 'vFRfuture_li': (['lat', 'lon'],  vFRfuture_li), 
                 'vFRfuture_li2': (['lat', 'lon'],  vFRfuture_li2), 
                 'vFRfuture_np': (['lat', 'lon'],  vFRfuture_np)                 
                },
                coords={'lat':lats,'lon':lons,'mod':ModelArray})

ds.to_netcdf('~/GoogleDrive/My/My.Research/UCI/Projects/Lightning/Results/df_boreal55_FRpredictions.nc')

***

### FR change as a function of distance to treeline

In [20]:
# Read distance to the treeline
dsdis = xr.open_dataset('/Users/yangchen/Jacaranda/ProjectData/Lightning/Dis2Treeline.nc')
dsdis = ma.masked_invalid(dsdis.distance.values)  # -3~3

In [21]:
# Extract modeled data and limit to boreal forest area
FRhistmodBFS = ma.masked_invalid(ma.masked_array(vFR,mask=np.invert(maskBFS)))
FRfuturemodBFS = ma.masked_invalid(ma.masked_array(vFRfuture_final,mask=np.invert(maskBFS)))
FRscalerstdBFS = ma.masked_invalid(ma.masked_array(vFRscaler_std,mask=np.invert(maskBFS)))
FRfuturemodallBFS = np.zeros((180,360,nmodel))
for imodel in range(nmodel):
    FRfuturemodallBFS[:,:,imodel] = ma.masked_invalid(ma.masked_array(vFRfuture_final_all[:,:,imodel],mask=np.invert(maskBFS)))
CxPhistmodBFS = ma.masked_invalid(ma.masked_array(cxphistmean,mask=np.invert(maskBFS)))
CxPfuturemodBFS = ma.masked_invalid(ma.masked_array(cxprcpmean,mask=np.invert(maskBFS))) 
CAPEhistmodBFS = ma.masked_invalid(ma.masked_array(capehistmean,mask=np.invert(maskBFS)))
CAPEfuturemodBFS = ma.masked_invalid(ma.masked_array(capercpmean,mask=np.invert(maskBFS))) 
TPhistmodBFS = ma.masked_invalid(ma.masked_array(tprhistmean,mask=np.invert(maskBFS)))
TPfuturemodBFS = ma.masked_invalid(ma.masked_array(tprrcpmean,mask=np.invert(maskBFS)))
CPhistmodBFS = ma.masked_invalid(ma.masked_array(prchistmean,mask=np.invert(maskBFS)))
CPfuturemodBFS = ma.masked_invalid(ma.masked_array(prcrcpmean,mask=np.invert(maskBFS)))
PSCBFS = ma.masked_invalid(ma.masked_array(PSC,mask=np.invert(maskBFS)))
vBABFS = ma.masked_invalid(ma.masked_array(vBA,mask=np.invert(maskBFS)))

In [ ]:
# Calculate mean values in bins of distance to treeline (DOT)
area1x1 = np.tile(np.cos(np.radians(np.linspace(-89.5,89.5,180)))*772.*16.,(360,1)).T  # km2/1deg, [180,360]
nbindis =   60
FRhistmodbin,FRfuturemodbin,FRfuturemodallbin = np.zeros(nbindis),np.zeros(nbindis),np.zeros((nbindis,nmodel))
FRstdbin = np.zeros(nbindis)
CxPhistmodbin,CxPfuturemodbin = np.zeros(nbindis),np.zeros(nbindis)
CAPEhistmodbin,CAPEfuturemodbin = np.zeros(nbindis),np.zeros(nbindis)
TPhistmodbin,TPfuturemodbin = np.zeros(nbindis),np.zeros(nbindis)
CPhistmodbin,CPfuturemodbin = np.zeros(nbindis),np.zeros(nbindis)
PSCbin = np.zeros(nbindis)
vBAbin = np.zeros(nbindis)
tPSCbin = np.zeros(nbindis)
areabin = np.zeros(nbindis)
for idis in range(nbindis):
    vtest = ma.masked_outside(dsdis+3,(idis*0.1),(idis*0.1+0.1))
#     vtest = ma.masked_outside(dsdis+3,(idis*0.05),(idis*0.05+0.05))    
    if ma.is_masked(vtest.sum())==False:
        dismask = (vtest.mask==False) * (maskBFS==1)
        FRhistmodbin[idis] = FRhistmodBFS[np.where(dismask==1)].mean()
        FRfuturemodbin[idis] = FRfuturemodBFS[np.where(dismask==1)].mean()
        
        FRstdbin[idis] = FRscalerstdBFS[np.where(dismask==1)].mean() * FRhistmodbin[idis]
        
        for imodel in range(nmodel):
            FRfuturemodsmBFS = ma.masked_invalid(FRfuturemodallBFS[:,:,imodel])
            FRfuturemodallbin[idis,imodel] = FRfuturemodsmBFS[np.where(dismask==1)].mean()
        CxPhistmodbin[idis] = CxPhistmodBFS[np.where(dismask==1)].mean()
        CxPfuturemodbin[idis] = CxPfuturemodBFS[np.where(dismask==1)].mean()
        CAPEhistmodbin[idis] = CAPEhistmodBFS[np.where(dismask==1)].mean()
        CAPEfuturemodbin[idis] = CAPEfuturemodBFS[np.where(dismask==1)].mean()  
        TPhistmodbin[idis] = TPhistmodBFS[np.where(dismask==1)].mean()
        TPfuturemodbin[idis] = TPfuturemodBFS[np.where(dismask==1)].mean() 
        CPhistmodbin[idis] = CPhistmodBFS[np.where(dismask==1)].mean()
        CPfuturemodbin[idis] = CPfuturemodBFS[np.where(dismask==1)].mean()         
        PSCbin[idis] = PSCBFS[np.where(dismask==1)].mean()
        vBAbin[idis] = vBABFS[np.where(dismask==1)].mean()     
        tPSCbin[idis] = (PSCBFS[np.where(dismask==1)]*area1x1[np.where(dismask==1)]).sum()/1e6   # Pg C/km
        areabin[idis] = (area1x1[np.where(dismask==1)]).sum()

need to save predicted binned values

In [26]:
bins = -(np.arange(60)*0.1-3+0.05)   # convert to unit of 'km north of treeline'
ds = xr.Dataset({'FRhistmodbin': (['bin'],  FRhistmodbin), 
                 'FRstdbin': (['bin'],  FRstdbin), 
                 'FRfuturemodbin': (['bin'],  FRfuturemodbin), 
                 'FRfuturemodallbin': (['bin', 'mod'], FRfuturemodallbin), 
                 'CxPhistmodbin': (['bin'],  CxPhistmodbin), 
                 'CxPfuturemodbin': (['bin'],  CxPfuturemodbin), 
                 'CAPEhistmodbin': (['bin'],  CAPEhistmodbin), 
                 'CAPEfuturemodbin': (['bin'],  CAPEfuturemodbin), 
                 'TPhistmodbin': (['bin'],  TPhistmodbin), 
                 'TPfuturemodbin': (['bin'],  TPfuturemodbin), 
                 'CPhistmodbin': (['bin'],  CPhistmodbin), 
                 'CPfuturemodbin': (['bin'],  CPfuturemodbin),
                 'PSCbin': (['bin'],  PSCbin),
                 'vBAbin': (['bin'],  vBAbin),
                 'tPSCbin': (['bin'],  tPSCbin),
                 'areabin': (['bin'],  areabin)                 
                },
                coords={'bin':bins,'mod':ModelArray})

In [27]:
ds.to_netcdf('~/GoogleDrive/My/My.Research/UCI/Projects/Lightning/Results/df_boreal55_FRpredictionsbins.nc')

***

## Predict future TCF

### Using the default linear model
using CAPE and TAS values from CMIP5:  
* hist - summer of 1986-2005 (historical)
* rcp -  summer of 2081-2100 (future)

method:
* Use historical all model mean CAPE and TAS and observed FR (FRpd) to derive the optimized a,b,c for TCF = a*CAPE+b*T+c
* For each CMIP5 model, calculate modeled TCF for 1986-2005 using the above derived parameters and CAPE+TAS:
    * TCFhist_i = a*CAPEhist_i+b*Thist_i+c
* Similarly, for each model, calculate modeled mean TCF for 2081-2100:
    * TCFrcp_i = a*CAPErcp_i+b*Trcp_i+c
* To derive the final estimation of TCFrcp, we use modeled change (obtained from the all-model mean TCFhist and TCFrcp) to add to observed TCF in present day:
    * TCFrcp_final = TCFhist_obs + mean(TCFrcp_i-TCFhist_i)

In [28]:
def Proj_TCF(a,b,c,capehistall,capercpall,tashistall,tasrcpall,vTCF):
    # first, apply the powerlaw model to historical and future meterology from each model
    vTCFhist_map_mean = np.zeros((180,360))
    vTCFfuture_map_mean = np.zeros((180,360))
    for imodel in range(nmodel):
        model = ModelArray[imodel]        
        # present day
        vTCFhist_map_mean += (a*capehistall[:,:,imodel] + b*tashistall[:,:,imodel] + c) 
        # future
        vTCFfuture_map_mean += (a*capercpall[:,:,imodel] + b*tasrcpall[:,:,imodel] + c) 
    vTCFhist_map_mean /= nmodel
    vTCFfuture_map_mean /= nmodel   
    
    # the final projection is using modeled FRfuture/FRhist to scale observed FR
    vTCFfuture_final = np.clip(vTCF + (vTCFfuture_map_mean-vTCFhist_map_mean),0,100)

        
    # return values
    return vTCFfuture_final

In [29]:
vtmp = pd.read_csv('~/GoogleDrive/My/My.Research/UCI/Projects/Lightning/Results/TCF.vs.CAPE_T.csv',header=None)
f_Intercept,f_CAPE,f_T = vtmp[1][:]

vTCFfuture_final = Proj_TCF(f_CAPE,f_T,f_Intercept,capehistall,capercpall,tashistall,tasrcpall,vTCF)
vTCFfuture_final = ma.masked_array(vTCFfuture_final,mask=(maskBFS==False))

In [30]:
ds = xr.Dataset({'vTCFhist_final': (['lat', 'lon'],  vTCF), 
                 'vTCFfuture_final': (['lat', 'lon'],  vTCFfuture_final),
                },
                coords={'lat':lats,'lon':lons})

ds.to_netcdf('~/GoogleDrive/My/My.Research/UCI/Projects/Lightning/Results/df_boreal55_TCFprediction.nc')

## Predict future BA

### Using the default logarithmatic model (ln(BA) ~ TCF + T + FR)
using TAS values from CMIP5, FR and TCF from observation and estimation:  
* hist - summer of 1986-2005 (historical)
* rcp -  summer of 2081-2100 (future)

method:
* Use historical all model mean TAS and observed FR, TCF, BA to derive the optimized a,b,c,d for BA = a * T+b * FR+c * TCF+d
* Use the model mean parameters to estimate future BA
    * BArcp_final = a * tasrcpmean+ b * vFRfuture_final +c * vTCFfuture_final+ d

In [31]:
vtmp = pd.read_csv('~/GoogleDrive/My/My.Research/UCI/Projects/Lightning/Results/lnBA.vs.T_FR_TCF.csv',header=None)
f_Intercept,f_TCF,f_T,f_FR = vtmp[1][:]

In [32]:
vBAfuture_final = ( np.exp(f_Intercept + f_TCF*vTCFfuture_final + f_T*tasrcpmean + f_FR*vFRfuture_final)/
                  np.exp(f_Intercept + f_TCF*vTCF + f_T*tashistmean + f_FR*vFR) ) * vBA

vBAfuture_final = ma.masked_array(vBAfuture_final,mask=(maskBFS==False))

In [33]:
ds = xr.Dataset({'vBAhist_final': (['lat', 'lon'],  vBA), 
                 'vBAfuture_final': (['lat', 'lon'],  vBAfuture_final),
                },
                coords={'lat':lats,'lon':lons})

ds.to_netcdf('~/GoogleDrive/My/My.Research/UCI/Projects/Lightning/Results/df_boreal55_BAprediction.nc')

## Analysis of mean values in different regions/ecosystems

### Read masks

In [34]:
# Set masks of TUNDRA (vTCF<1) and FOREST (vTCF >= 1)

maskTUNDRA = ((vTCF<1)*maskBFS).data
maskFOREST = ((vTCF>=1)*maskBFS).data

mask_NA = (np.zeros((180,360))!=0)
mask_NA[:,:150] = True

maskBFS_NA = mask_NA * maskBFS
maskBFS_EA = (mask_NA==False) * maskBFS

maskTUNDRA_NA = mask_NA * maskTUNDRA
maskTUNDRA_EA = (mask_NA==False) * maskTUNDRA

maskFOREST_NA = mask_NA * maskFOREST
maskFOREST_EA = (mask_NA==False) * maskFOREST

damaskBFS = xr.DataArray(maskBFS,coords={'lat':lats,'lon':lons}, dims=['lat','lon'])

### Calculate mean values

In [35]:
# function used to calculate means
def Paramean(data):
    data = ma.masked_invalid(data)
    
    dataALL = ma.masked_array(data,mask=(maskBFS==0)).mean()
    dataALL_NA = ma.masked_array(data,mask=(maskBFS_NA==0)).mean()
    dataALL_EA = ma.masked_array(data,mask=(maskBFS_EA==0)).mean()    
    dataTundra = ma.masked_array(data,mask=(maskTUNDRA==0)).mean()
    dataTundra_NA = ma.masked_array(data,mask=(maskTUNDRA_NA==0)).mean()
    dataTundra_EA = ma.masked_array(data,mask=(maskTUNDRA_EA==0)).mean()
    dataForest = ma.masked_array(data,mask=(maskFOREST==0)).mean()
    dataForest_NA = ma.masked_array(data,mask=(maskFOREST_NA==0)).mean()
    dataForest_EA = ma.masked_array(data,mask=(maskFOREST_EA==0)).mean()

    dataPSCwgt = ma.average(data,weights=maskBFS*vPSC)
    dataPSCwgt_NA = ma.average(data,weights=maskBFS_NA*vPSC)
    dataPSCwgt_EA = ma.average(data,weights=maskBFS_EA*vPSC)
    
    return dataALL,dataALL_NA,dataALL_EA,dataTundra,dataTundra_NA,dataTundra_EA, \
           dataForest,dataForest_NA, dataForest_EA, dataPSCwgt,dataPSCwgt_NA,dataPSCwgt_EA

### Derive the table

In [36]:
FRtable = pd.DataFrame(columns=['All','All_NA','All_EA','Tundra','Tundra_NA','Tundra_EA',
                                'Forest','Forest_NA','Forest_EA','PSCwgt','PSCwgt_NA','PSCwgt_EA'])

In [37]:
# historical
FRtable.loc['BA']=Paramean(vBA)                    # %/yr
FRtable.loc['PSC']=Paramean(vPSC)                  # kg/m2
FRtable.loc['CxPhist']=Paramean(cxphistmean*1000)   # 10^3 W/m2
FRtable.loc['CAPEhist']=Paramean(capehistmean)            # J/kg
FRtable.loc['TPhist']=Paramean(tprhistmean)            # mm/day
FRtable.loc['CPhist']=Paramean(prchistmean)            # mm/day
FRtable.loc['FRhist']=Paramean(vFR)              # #/km2/m

# projected future from pl approach
FRtable.loc['CxPfuture']=Paramean(cxprcpmean*1000)   # 10^3 W/m2
FRtable.loc['CAPEfuture']=Paramean(capercpmean)            # J/kg
FRtable.loc['TPfuture']=Paramean(tprrcpmean)            # mm/day
FRtable.loc['CPfuture']=Paramean(prcrcpmean)            # mm/day
FRtable.loc['FRfuture']=Paramean(vFRfuture_final)    # #/km2/m
FRtable.loc['FRfuturedC']=Paramean(vFRfuturedC_final)    # #/km2/m
FRtable.loc['FRfuturedP']=Paramean(vFRfuturedP_final)    # #/km2/m

# projected change from pl approach
FRtable.loc['CxPchange']=Paramean(cxprcpmean*1000-cxphistmean*1000) # 10^3 W/m2
FRtable.loc['CAPEchange']=Paramean(capercpmean-capehistmean)        # J/kg
FRtable.loc['TPchange']=Paramean(tprrcpmean-tprhistmean)            # mm/day
FRtable.loc['CPchange']=Paramean(prcrcpmean-prchistmean)            # mm/day
FRtable.loc['FRchange']=Paramean(vFRfuture_final-vFR)           # #/km2/m
FRtable.loc['FRchangedC']=Paramean(vFRfuturedC_final-vFR)           # #/km2/m
FRtable.loc['FRchangedP']=Paramean(vFRfuturedP_final-vFR)           # #/km2/m

# projected change from pl approach (relative)
FRtable.loc['CxPchange_r']=FRtable.loc['CxPchange']/FRtable.loc['CxPhist']*100 # %
FRtable.loc['CAPEchange_r']=FRtable.loc['CAPEchange']/FRtable.loc['CAPEhist']*100 # %
FRtable.loc['TPchange_r']=FRtable.loc['TPchange']/FRtable.loc['TPhist']*100 # %
FRtable.loc['CPchange_r']=FRtable.loc['CPchange']/FRtable.loc['CPhist']*100 # %
FRtable.loc['FRchange_r']=FRtable.loc['FRchange']/FRtable.loc['FRhist']*100 # %
FRtable.loc['FRchangedC_r']=FRtable.loc['FRchangedC']/FRtable.loc['FRhist']*100 # %
FRtable.loc['FRchangedP_r']=FRtable.loc['FRchangedP']/FRtable.loc['FRhist']*100 # %

# projected future from other approaches
FRtable.loc['FRfuture_pl_op']=Paramean(vFRfuture_pl_op) 
FRtable.loc['FRfuture_sc']=Paramean(vFRfuture_sc) 
FRtable.loc['FRfuture_li']=Paramean(vFRfuture_li) 
FRtable.loc['FRfuture_li2']=Paramean(vFRfuture_li2) 
FRtable.loc['FRfuture_np']=Paramean(vFRfuture_np) 

# projected change from other approaches
FRtable.loc['FRchange']= FRtable.loc['FRfuture'] - FRtable.loc['FRhist']
FRtable.loc['FRchange_pl_op']= FRtable.loc['FRfuture_pl_op'] - FRtable.loc['FRhist']
FRtable.loc['FRchange_sc']= FRtable.loc['FRfuture_sc'] - FRtable.loc['FRhist']
FRtable.loc['FRchange_li']= FRtable.loc['FRfuture_li'] - FRtable.loc['FRhist']     
FRtable.loc['FRchange_li2']= FRtable.loc['FRfuture_li2'] - FRtable.loc['FRhist']
FRtable.loc['FRchange_np']= FRtable.loc['FRfuture_np'] - FRtable.loc['FRhist']

# projected change from other approaches (relative)
FRtable.loc['FRchange_r']=FRtable.loc['FRchange']/FRtable.loc['FRhist']*100 # %
FRtable.loc['FRchange_pl_op_r']=FRtable.loc['FRchange_pl_op']/FRtable.loc['FRhist']*100 # %
FRtable.loc['FRchange_sc_r']=FRtable.loc['FRchange_sc']/FRtable.loc['FRhist']*100 # %
FRtable.loc['FRchange_li_r']=FRtable.loc['FRchange_li']/FRtable.loc['FRhist']*100 # %
FRtable.loc['FRchange_li2_r']=FRtable.loc['FRchange_li2']/FRtable.loc['FRhist']*100 # %
FRtable.loc['FRchange_np_r']=FRtable.loc['FRchange_np']/FRtable.loc['FRhist']*100 # %

In [38]:
FRtable2 = FRtable.applymap('{:.2f}'.format)
# FRtable.style.format({
#     'All': '{:.2f}'.format,'All_NA': '{:.2f}'.format,'All_EA': '{:.2f}'.format,
#     'Tundra': '{:.2f}'.format, 'Tundra_NA': '{:.2f}'.format,'Tundra_EA': '{:.2f}'.format, 
#     'Forest': '{:.2f}'.format, 'Forest_NA': '{:.2f}'.format,'Forest_EA': '{:.2f}'.format,
#     'PSCwgt': '{:.2f}'.format, 'PSCwgt_NA': '{:.2f}'.format,'PSCwgt_EA': '{:.2f}'.format    
# })

### Show the tables

Mean values at contemporary period

In [39]:
FRtable2.loc[['FRhist','CxPhist','CAPEhist','TPhist','CPhist','BA','PSC']]

,All,All_NA,All_EA,Tundra,Tundra_NA,Tundra_EA,Forest,Forest_NA,Forest_EA,PSCwgt,PSCwgt_NA,PSCwgt_EA
FRhist,0.23,0.14,0.27,0.07,0.05,0.08,0.33,0.20,0.38,0.18,0.16,0.19
CxPhist,6.58,6.53,6.61,3.10,3.61,2.86,8.62,8.47,8.68,5.32,6.40,4.95
CAPEhist,119.17,114.05,121.32,63.19,70.25,59.87,151.84,143.07,155.27,100.41,115.12,95.36
TPhist,1.96,1.99,1.95,1.67,1.68,1.66,2.13,2.20,2.10,1.82,1.80,1.83
CPhist,0.64,0.65,0.63,0.40,0.44,0.38,0.78,0.79,0.77,0.55,0.61,0.53
BA,0.27,0.32,0.24,0.10,0.06,0.11,0.37,0.49,0.32,0.27,0.34,0.25
PSC,24.62,17.52,28.60,25.63,15.99,30.22,23.77,18.61,27.06,34.40,33.88,34.58


Mean values at future

In [40]:
FRtable2.loc[['FRfuture','CxPfuture','CAPEfuture','TPfuture','CPfuture','FRfuturedC','FRfuturedP']]

,All,All_NA,All_EA,Tundra,Tundra_NA,Tundra_EA,Forest,Forest_NA,Forest_EA,PSCwgt,PSCwgt_NA,PSCwgt_EA
FRfuture,0.53,0.28,0.63,0.20,0.13,0.23,0.72,0.38,0.85,0.45,0.32,0.49
CxPfuture,12.30,11.42,12.67,6.66,6.95,6.52,15.60,14.39,16.08,10.47,11.27,10.19
CAPEfuture,207.49,184.22,217.25,119.69,120.08,119.52,258.72,226.71,271.26,180.95,187.41,178.73
TPfuture,2.27,2.34,2.24,2.01,2.03,2.00,2.43,2.54,2.38,2.14,2.12,2.14
CPfuture,0.87,0.85,0.87,0.58,0.61,0.57,1.03,1.02,1.04,0.77,0.80,0.75
FRfuturedC,0.48,0.26,0.58,0.16,0.11,0.19,0.67,0.36,0.80,0.40,0.29,0.43
FRfuturedP,0.27,0.16,0.32,0.09,0.07,0.09,0.38,0.22,0.45,0.22,0.18,0.23


Mean changes from contemporary period to future

In [41]:
FRtable2.loc[['FRchange','CxPchange','CAPEchange','TPchange','CPchange','FRchangedC','FRchangedP']]

,All,All_NA,All_EA,Tundra,Tundra_NA,Tundra_EA,Forest,Forest_NA,Forest_EA,PSCwgt,PSCwgt_NA,PSCwgt_EA
FRchange,0.29,0.14,0.36,0.13,0.07,0.15,0.39,0.19,0.47,0.26,0.16,0.30
CxPchange,5.72,4.89,6.07,3.56,3.34,3.66,6.98,5.92,7.40,5.15,4.87,5.25
CAPEchange,88.32,70.17,95.94,56.51,49.83,59.65,106.88,83.64,115.99,80.54,72.29,83.38
TPchange,0.31,0.34,0.30,0.34,0.35,0.33,0.30,0.34,0.28,0.32,0.32,0.31
CPchange,0.23,0.20,0.24,0.18,0.17,0.19,0.25,0.22,0.27,0.22,0.19,0.23
FRchangedC,0.25,0.12,0.31,0.09,0.05,0.11,0.34,0.16,0.42,0.22,0.14,0.24
FRchangedP,0.04,0.02,0.05,0.02,0.01,0.02,0.05,0.03,0.07,0.03,0.03,0.04


Mean percent changes

In [42]:
FRtable2.loc[['FRchange_r','CxPchange_r','CAPEchange_r','TPchange_r','CPchange_r','FRchangedC_r','FRchangedP_r']]

,All,All_NA,All_EA,Tundra,Tundra_NA,Tundra_EA,Forest,Forest_NA,Forest_EA,PSCwgt,PSCwgt_NA,PSCwgt_EA
FRchange_r,126.51,102.73,131.57,182.15,142.22,194.72,119.64,95.78,124.45,143.70,105.59,154.29
CxPchange_r,86.91,74.92,91.88,114.94,92.61,128.19,81.03,69.92,85.28,96.86,76.04,106.10
CAPEchange_r,74.11,61.52,79.08,89.43,70.93,99.63,70.39,58.46,74.70,80.22,62.80,87.44
TPchange_r,15.87,17.19,15.30,20.27,20.94,19.95,13.86,15.30,13.27,17.43,18.01,17.23
CPchange_r,35.60,30.65,37.75,45.71,37.63,50.13,32.57,28.08,34.38,40.20,31.94,43.48
FRchangedC_r,108.19,86.95,112.71,133.64,105.65,142.46,105.05,83.66,109.36,117.82,88.30,126.03
FRchangedP_r,17.27,15.55,17.63,22.42,25.59,21.42,16.63,13.78,17.20,18.51,16.52,19.07


In [43]:
# FRtable2.loc[['FRfuture','FRfuture_pl_op','FRfuture_sc','FRfuture_li','FRfuture_li2','FRfuture_np']]

Mean percent changes from different regression models

In [44]:
FRtable2.loc[['FRchange_r','FRchange_pl_op_r','FRchange_sc_r','FRchange_li_r','FRchange_li2_r','FRchange_np_r']]

,All,All_NA,All_EA,Tundra,Tundra_NA,Tundra_EA,Forest,Forest_NA,Forest_EA,PSCwgt,PSCwgt_NA,PSCwgt_EA
FRchange_r,126.51,102.73,131.57,182.15,142.22,194.72,119.64,95.78,124.45,143.70,105.59,154.29
FRchange_pl_op_r,135.40,109.45,140.94,196.48,152.62,210.30,127.87,101.85,133.11,154.15,112.42,165.74
FRchange_sc_r,86.30,71.78,89.40,120.06,96.22,127.57,82.14,67.47,85.10,97.07,74.01,103.48
FRchange_li_r,109.66,157.17,99.53,220.00,276.90,202.08,96.04,136.08,87.97,124.86,138.96,120.94
FRchange_li2_r,167.71,240.61,152.17,307.47,401.41,277.88,150.46,212.29,137.99,187.37,212.44,180.40
FRchange_np_r,115.90,90.45,121.29,185.72,128.50,203.63,107.25,83.63,112.01,137.50,94.33,149.46


***

## Statistics for four models

In [53]:
dfstats = pd.read_csv('~/GoogleDrive/My/My.Research/UCI/Projects/Lightning/Results/dfstats_boreal55_summer1996-1999_FRmod.csv',index_col=0)
FRtable1 = dfstats[['r2','slope','bias','biase(s)','rmse','rmse(s)']].loc[['power law','scale','linear type I','non-para']].applymap('{:.2f}'.format)

In [56]:
FRtable3 = FRtable2[['All','Tundra','Forest','PSCwgt']].loc[['FRchange_r','FRchange_sc_r','FRchange_li_r','FRchange_np_r']]
FRtable3.index=['power law', 'scale', 'linear type I', 'non-para']
FRtable3.columns = ['dFR_All', 'dFR_Tundra', 'dFR_Forest', 'dFR_PSCwgt']

In [57]:
pd.concat([FRtable1,FRtable3],axis=1)

,r2,slope,bias,biase(s),rmse,rmse(s),dFR_All,dFR_Tundra,dFR_Forest,dFR_PSCwgt
power law,0.63,0.50,-22.82,22.28,0.15,0.08,126.51,182.15,119.64,143.70
scale,0.63,0.52,9.05,105.66,0.16,0.10,86.30,120.06,82.14,97.07
linear type I,0.63,0.61,1.44,65.78,0.19,0.12,109.66,220.00,96.04,124.86
non-para,0.65,0.65,0.15,55.00,0.20,0.11,115.90,185.72,107.25,137.50
